## Theoretical Formulation

### A. Demand
Let $m$ := be the month of the year where $m \in M$ where $M = \{{1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12}\}$

Let $h$ := be the hour of the day where $h \in H$ where $H = \{{0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23}\}$

Let $d_{m,h}$:= be the customer electricity demand in KwH during hour $h$ in month $m$  where $d_{m,h} \geq 0$

### B. Power Source Parameters
Let $j$ := be a product $j$ $\in$ $J$ where $J = \{{"smallsolar", "largesolar", "smallwind", "largewind", "smallbattery"}\}$

Let $E$ := be a set containing indices of products which generate electricity 

Let $S$ := be a set containing indices of products which store electricity 

Let $et$ := be an energy type where $et \in T = \{{"solar", "wind", "storage"}\}$

Let $ET_{et}$ := be a set containing all products of energy type $et$

Let $f: J \rightarrow ET_{et}$ := be a dictionary that maps a product $j$ to the set containing all products with the same energy type $et$

Let $oc_{j}$ := be the opening cost in US dollars of installing required infrastructure for product $j$ which belongs to an energy type $et$

Let $ic_j$ := be the incremental cost in US dollars of opening up an additional unit of a specific product $j$ where $ic_j \geq 0$

Let $mc_j$ := be the monthly cost in US dollars of maintaining one unit of product $j$ where $mc_j \geq 0$

Let $\rho_{m,h,j}$ := be the efficiency of product $j$ during hour $h$ in month $m$ where $\rho_{m,h,j}$ is continuous from 0 to 1

Let $c_{j}$ := be the capacity of product $j$ in KwH if it generates or stores electricity

Let $gc_{m,h}$ := be the cost of on-grid power in units of dollers per kilowatt hour

Let $gp_{m,h}$ := be the price of energy when you sell to the grid in units of dollers per kilowatt hour

### C. Decision Variables
Let $x_j$ := be a binary variable representing whether or not the opening cost of $j$'s energy type is paid for where $x_j \in$ {0,1}

Let $y_{j}$ := be an integer variable representing the number of units of product $j$ where $y_j \geq$ 0

Let $g_{m,h}$ := be the continuous variable that represents the amount of power needed from the grid in kilowatt-hours during hour $h$ in month $m$.

Let $gi_{m,h}$ := be the binary variable that represents whether the project is connected to the grid.

Let $es_{m,h,j}$ := be the continuous variable that represents the amount of energy stored by product $j$ (note: not all products can store energy)

Let $sc_{m,h,j}$ := be the continuous variable that represents the amount of stored consumed from product $j$ (note: not all products can store energy)

Let $ss_{m,h}$ := be the continuous variable that represents the amount of energy sold during hour $h$ in month $m$

### D. Calculated Variables

Let $ES_{m,h}$ := be the total energy stored this year through month $m$ and hour $h$ calculated by summing cumulative the energy stored each hour for all products $\sum_{j \in J} es_{m,h,j}$

Let $SC_{m,h}$ := be the total storage consumed this year through month $m$ and hour $h$ calculated by summing the cumulative energy consumed each hour for all products $\sum_{j \in J} sc_{m,h,j}$

Let $TT$ := be the total energy storage of the system calculated by summing the capacity of all products $\sum_{j \in J} c_{j}y_{j}$

Let $SS_{m,h}$ := be the total energy sold this year through month $m$ and hour $h$ calculated by summing the cumulative energy sold each hour $\sum ss_{m,h}$

Let $TR$ := be the total revenue this year calculated by summing energy sold each hour $\sum ss_{m,h}gp_{m,h}$


### E. Objective Function
We want to minimize the cost; therefore, the objective function is the sum of the three kinds of costs:

    a) opening cost (oc) the cost of initially setting up a new service

    b) incremental cost (ic)
    
    c) monthly cost (mc)

    d) grid monthly cost (g)

$$\min \sum_{j \in J} oc_{j}x_j + \sum_{j \in J} ic_jy_j + \sum_{j \in J} mc_{j}y_{j}  + \sum_{m \in M} \sum_{h \in H} g_{m,h} + gi * -TR + (1 - gi) * RT$$ 

### F. Constraints

##### Demand Constraints
1) Meet customer demand for electricity per hour per month by summing the energy generation products ($E$) output, the stored energy ($S$) available, and grid power ($g$) available

$$\forall m \in M, \forall h \in H:  \sum_{j\in E} \rho_{m,h,j} y_{j} + \sum_{j\in S}sc_{m,h,j} + g_{m,h} \geq d_{m,h}$$

##### Budget Constraints
2) You have an initial budget of the amount of capital you have on hand to shell out for initial set up.
$$  \sum_{j \in J} oc_{j}x_j + \sum_{j \in J} ic_jy_j \leq $200,000 $$

3) You also have a monthly budget.
$$ \sum_{j \in J} mc_{j}y_{j} \leq $500 $$

##### Set up and Installation Constraints
4) Ensure at least 1 $x_j$ per energy type is 1 when any $y_j$ of the energy type is $\geq 1$

$$ \forall j \in ET_{solar}: \sum_{j' \in ET_{solar}} x_{j'} * M >= y_{j}$$

$$ \forall j \in ET_{wind}: \sum_{j' \in ET_{wind}} x_{j'} * M >= y_{j}$$

$$ \forall j \in ET_{storage}: \sum_{j' \in ET_{storage}} x_{j'} * M >= y_{j}$$

$$ gi * M >= \sum gc_{m,h}$$


5) Ensure that at most 1 $x_j$ per energy type is equal to 1

$$ \forall et \in ET: \sum_{j\in ET_{et}}x_j \leq 1$$

##### Storage Constraints
6) Total storage consumed at hour $h$ during month $m$ cannot exceed electricity stored

$$ \forall m \in M, \forall h \in H\space: SC_{m,h} + SS_{m,h} \leq ES_{m,h} $$ 

7) Storage available at hour $h$ during month $m$ cannot exceed total storage capacity

$$ \forall m \in M, \forall h \in H\space: ES_{m,h} - SC_{m,h} - SS_{m,h} \leq TT $$

8) Electricity stored at hour $h$ during month $m$ cannot exceed the sum of electricity produced, storage consumed, and grid usage minus demand at hour $h$ during month $m$ 

$$ \forall m \in M, \forall h \in H:\space \sum_{j\in S} es_{m,h,j} \leq \sum_{j\in E} \rho_{m,h,j} y_{j} + \sum_{j\in S} sc_{m,h,j} + g_{m,h} - d_{m,h} $$

